## Homework 2 Answer

Grading:

1- 5 pts

2- 6 pts

3- 3 pts (0.5 each)

4- 6 pts

This homework covers the material in Lessons 3 & 4.  It is due **Thurday, June 15**.  Please submit your **answers as a PDF**.

1\. Modify the LU method (without pivoting) to work **in-place**.  That is, it should not allocate any new memory for L or U, but instead overwrite A.

In [2]:
import numpy as np
def LU(A):
    U = np.copy(A)
    m, n = A.shape
    L = np.eye(n)
    for k in range(n-1):
        for j in range(k+1,n):
            L[j,k] = U[j,k]/U[k,k]
            U[j,k:n] -= L[j,k] * U[k,k:n]
    return L, U

In [24]:
# Answer
# by the way, this LU only works for a square matrix
def LU_InPlace(A):
    m, n = A.shape
    for k in range(n-1):
        for j in range(k+1,n):
            A[j,k] = A[j,k]/A[k,k]
            A[j,(k+1):n] -= (A[j,k]) * A[k,(k+1):n]
    return A
# This one single matrix A will store both L and U. 
# The upper part of A is U. The lower part of A (without the diagonal) is L. diagonal of L is always 1's.

In [23]:
# verify
A = np.array([[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]).astype(np.float)
L, U = LU(A)
A = LU_InPlace(A)
print(L,U,A)

[[ 1.  0.  0.  0.]
 [ 2.  1.  0.  0.]
 [ 4.  3.  1.  0.]
 [ 3.  4.  1.  1.]] [[ 2.  1.  1.  0.]
 [ 0.  1.  1.  1.]
 [ 0.  0.  2.  2.]
 [ 0.  0.  0.  2.]] [[ 2.  1.  1.  0.]
 [ 2.  1.  1.  1.]
 [ 4.  3.  2.  2.]
 [ 3.  4.  1.  2.]]


2\. Modify our LU method from class to add pivoting, as described in the lesson.  *Hint: the swap method below will be useful* 

In [44]:
# Anaswer. Again this will only work for square
import numpy as np
def LU_pivot(A):
    
    m, n = A.shape
    L = np.eye(n)
    P = np.eye(n)
    RowInd = [] #Initiate an empty list for row indices
    for j in range(n-1): #column j
        # Find the row index of the largest (in magnitude) entry
        AllRI = list(set(range(n)) - set(RowInd))
        AllRV = A[AllRI,j]
        fi = np.argmax(AllRV)
        ri = AllRI[fi]
        RowInd.append(ri) # update RowInd

        for i in list(set(AllRI) - set([ri])):
            c = -A[i,j]/A[ri,j]
            A[i,j:n] = A[ri, j:n]*c + A[i,j:n]
            if c !=0:
                L[i,ri]=-c
    # building P L according to RowInd
    LastL = list(set(range(n)) - set(RowInd))
    LastI = LastL[0]
    RowInd.append(LastI)
    P = P[:,RowInd]
    U = A[RowInd,:]
    L = L[:,RowInd]
    L = L[RowInd,:]
    return P, L, U

In [47]:
# check
A = np.array([[2,1,1,0],[4,3,3,1],[8,7,9,5],[6,7,9,8]]).astype(np.float)
P,L,U = LU_pivot(A)
print(P,L,U)
print(P@L@U)

[[ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]] [[ 1.          0.          0.          0.        ]
 [ 0.75        1.          0.          0.        ]
 [ 0.25       -0.42857143  1.          0.        ]
 [ 0.5        -0.28571429  3.          1.        ]] [[ 8.          7.          9.          5.        ]
 [ 0.          1.75        2.25        4.25      ]
 [ 0.          0.         -0.28571429  0.57142857]
 [ 0.          0.          0.         -2.        ]]
[[ 2.  1.  1.  0.]
 [ 4.  3.  3.  1.]
 [ 8.  7.  9.  5.]
 [ 6.  7.  9.  8.]]


3\. For each of the following sets of dimensions, either
  - give the dimensions of the output of an operation on A and B, **or** 
  - answer *incompatible* if the dimensions are incompatible according to the [numpy rules of broadcasting](https://docs.scipy.org/doc/numpy-1.10.0/user/basics.broadcasting.html).

    a.  A      (2d array):  3 x 3
        B      (1d array):      1
        3 x 3

    b.  A      (2d array):      2 x 1
        B      (3d array):  6 x 4 x 2
        incompatible

    c.  A      (2d array):  5 x 4
        B      (1d array):      4
        5 x 4

    d.  A      (3d array):  32 x 64 x 8
        B      (3d array):   32 x 1 x 8
        32 x 64 x 8

    e.  A      (3d array):       64 x 1
        B      (3d array):  32 x 1 x 16
        32 x 64 x 16

    f.  A      (3d array):  32 x 64 x 2
        B      (3d array):   32 x 1 x 8
        incompatible

4\. Write how this matrix would be stored in compressed row format:

\begin{pmatrix}
  1 & & 0& & -2 & -3 \\
  & 3 & 0 & & & -9 \\
  &  & 0 & -7 & 4 & \\ 
  -1 & 2 & 0& 7 & & \\
  -3 & &0 & 26 & &
 \end{pmatrix}

This is a 5 by 6 matrix. the 3rd column is a zero column. Maybe not so obvious, so I filled in the zeros

|Index|0|1|2|3|4|5|6|7|8|9|10|11|
|:--- |
|Val|1|-2|-3|3|-9|-7|4|-1|2|7|-3|26|
|RowPtr|0|3|5|7|10|
|Col|0|4|5|1|5|3|4|0|1|3|0|3|